In [7]:
!pip install pgmpy

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# ---------------------------------------------------------
# 1. Define Bayesian Network Structure
# ---------------------------------------------------------
model = DiscreteBayesianNetwork([
    ('Burglary', 'Alarm'),
    ('Earthquake', 'Alarm'),
    ('Alarm', 'JohnCalls')
])


In [8]:
# 2. Define CPDs (Conditional Probability Tables)

# P(Burglary)
cpd_burglary = TabularCPD(
    variable='Burglary',
    variable_card=2,
    values=[[0.999], [0.001]]
)

# P(Earthquake)
cpd_earthquake = TabularCPD(
    variable='Earthquake',
    variable_card=2,
    values=[[0.998], [0.002]]
)

# P(Alarm | Burglary, Earthquake)
cpd_alarm = TabularCPD(
    variable='Alarm',
    variable_card=2,
    values=[
        [0.999, 0.71, 0.06, 0.05],  # Alarm = False
        [0.001, 0.29, 0.94, 0.95]   # Alarm = True
    ],
    evidence=['Burglary', 'Earthquake'],
    evidence_card=[2, 2]
)

# P(JohnCalls | Alarm)
cpd_john = TabularCPD(
    variable='JohnCalls',
    variable_card=2,
    values=[
        [0.95, 0.10],  # JohnCalls = False
        [0.05, 0.90]   # JohnCalls = True
    ],
    evidence=['Alarm'],
    evidence_card=[2]
)

# Add CPDs to model
model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_john)

# Validate the model
model.check_model()



True

In [9]:
# 3. Perform Bayesian Inference
infer = VariableElimination(model)

# Example Query:
# Likelihood of Burglary given JohnCalls = True
result = infer.query(
    variables=['Burglary'],
    evidence={'JohnCalls': 1}   # 1 = True
)

print("P(Burglary | JohnCalls = True):")
print(result)

P(Burglary | JohnCalls = True):
+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.9837 |
+-------------+-----------------+
| Burglary(1) |          0.0163 |
+-------------+-----------------+
